<a href="https://colab.research.google.com/github/ovapichi/biomecanica-datos/blob/celda_de_cargas/Celda_por_equipo_Armado_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fpdf


In [26]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
import fpdf as FPDF
from datetime import datetime
from fpdf import FPDF
import shutil
pdf = PDF("p","cm","A4")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#encabezado y pie de página del pdf
class PDF (FPDF):
  def header(self):
    self.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/logo unsam.png",2,0.5,2.5) #Cambiar Logos
    self.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/logo.png",15,0.5,2.5)
    self.set_font("arial","B",12)
    self.set_text_color(50,175,175)
    self.set_font("arial","B",10)
    self.ln(12)
  def footer(self):
    self.set_y(-1)
    self.set_font("arial","",12)
    self.set_text_color(0,0,0)
    self.cell(0,1,f'{self.page_no()}',align="R")
  def set_col(self, col):
  # Set column position:
    self.col = col
    x = 0.50 + col * 2.5
    self.set_left_margin(x)
    self.set_x(x)
#Carátula del pdf
def caratula(nombre,protocolo):
  now = datetime.now()
  hoy=now.date()
  profesionales= ["Lic. Toscano, Ana Paula",'Lic. García Piccinini, Osvaldo']
  pdf.set_y(4)
  #pdf.set_text_color(50,175,175)
  pdf.set_font("arial","",12)
  pdf.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/informe.png",12,5,10)
  pdf.cell(0,5,f'Dirigido a: {nombre}' ,ln=2, align='l')
  pdf.multi_cell(h=2, align='l', w=10, txt=f'Protocolo: {protocolo}\n')
  pdf.cell(0,5,f'{hoy}',ln=True, align='l')
  pdf.add_page()
  pdf.set_y(4)
  pdf.set_text_color(0,0,0)
  pdf.cell(2,2,f'Equipo profesional',ln=True, align='l')
  for crack in profesionales:
    pdf.cell(2,2,f'{crack}',ln=True, align='l')
  pdf.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/firmas lab.jpg",5.5,16,10)
  pdf.set_font("arial","",10)
  pdf.multi_cell(h=0.5, align='L', w=0, txt='El equipo responsable del Laboratorio de Biomecánica Análisis Funcional del Movimiento y Rendimiento Humano queda a su disposición para el análisis conjunto del caso, su interpretación, tanto como las eventuales consideraciones de las conclusiones. \nAgradecemos la confianza al permitirnos realizar esta evaluación. \nLo saludamos cordialmente.')
  return
  # =================================BLOQUE DE FUNCIONES=================================================



In [ ]:

  #CELDA DE CARGAS WIN LABORAT
  #INGRESAR CARPETA DE ARCHIVOS .dat
direccion_crudos="/content/drive/Shareddrives/Sociedad Hebraica Argentina/Noviembre 2023/Datos/Celda/Crudos/"
direccion_final="/content/drive/Shareddrives/Sociedad Hebraica Argentina/Noviembre 2023/Datos/Celda/"
fecha="Noviembre 2023"
df_final = pd.DataFrame()
df_individual = pd.DataFrame(index=[0],columns='cuad_der cuad_izq isq_der isq_izq'.split())
lista=[]
#Entrega un .csv con formato de columnas ["jugadora","maximo","musculo","tiempo"]

  def leer_carpeta(direccion):
    lista_archivos=[]
    for nombre_archivo in os.listdir(direccion):
      lista_archivos.append(nombre_archivo)
    lista_archivos.sort()
    #print(lista_archivos)
    return lista_archivos

  def seleccion_muslo(lista,direccion):
    jugadora=[]
    maximos=[]
    musculo=[]

    for archivo in lista:

      df=pd.read_csv(direccion+"/"+archivo,on_bad_lines='skip',sep = "\t", decimal=",",header=None, encoding="utf-16")
      df=df.drop([0],axis=0)
      df=df.drop([0,1,2], axis=1)
      maximos.append(df[3].max())
      max=df[3].max()
      #print(max)

      if archivo[-12:-4]=="Cuad Der":
        musculo.append("Cuadriceps Derechos")
        jugadora.append(archivo[:-13])
        df_individual['cuad_der'][0]=max
      elif archivo[-12:-4]=="Cuad Izq":
        musculo.append("Cuadriceps Izquierdos")
        jugadora.append(archivo[:-13])
        df_individual['cuad_izq'][0]=max
      elif archivo[-12:-4]==" Isq Der":
        jugadora.append(archivo[:-12])
        musculo.append("Isquios Derechos")
        df_individual['isq_der'][0]=max
      elif archivo[-12:-4]==" Isq Izq":
        jugadora.append(archivo[:-12])
        musculo.append("Isquios Izquierdo")
        df_individual['isq_izq'][0]=max
        df_individual.to_csv(direccion_final+"Procesados/"+archivo[:-12]+".csv")


    df_final['jugadora']=jugadora
    df_final['maximos']=maximos
    df_final['musculo']=musculo
    df_final['tiempo']=5
    return




lista=leer_carpeta(direccion_crudos)
seleccion_muslo(lista,direccion_crudos)

df_final.to_csv(direccion_final+"Celda"+fecha+".csv")
df_final

In [ ]:
#setear todo lo necesario



In [45]:

#función para la aplicación de fórmulas de asimetría


def procesar_maximos (cuad_der,cuad_izq,isq_der,isq_izq,nombre):
    relacionFlex=(isq_der/isq_izq)*100-100
    simetricoFlex=0
    asimetricoDFlex=0
    asimetricoIFlex=0
    simetricoFlex=(relacionFlex <=10) & (relacionFlex >=-10)
    asimetricoDFlex=(relacionFlex <-10) & (relacionFlex >=-25)
    asimetricoIFlex=(relacionFlex >10) & (relacionFlex <25)
    #relacion extensores
    simetricoExt=0
    asimetricoDExt=0
    asimetricoIExt=0

    relacionExt=(cuad_der/cuad_izq)*100-100

    simetricoExt=(relacionExt <=10) & (relacionExt >=-10)
    asimetricoDExt=(relacionExt <-10) & (relacionExt >=-25)
    asimetricoIExt=(relacionExt >10) & (relacionExt <25)

    #relacion interpierna derecha
    criticoD=(cuad_der+isq_der)*0.06
    sCriticoD=(cuad_der+isq_der)*0.3
    relacionMd=(cuad_der+isq_der)*0.7-cuad_der

    simetricoMd=0
    asimetricoDIsq=0
    asimetricoDCuad=0
    sAsimetricoDIsq=0
    sAsimetricoDCuad=0

    simetricoMd=(relacionMd <=criticoD) & (relacionMd >=criticoD*-1)
    asimetricoDIsq=(relacionMd <criticoD*-1) & (relacionMd >=sCriticoD*-1)
    asimetricoDCuad=(relacionMd >criticoD) & (relacionMd <sCriticoD)
    sAsimetricoDIsq=(relacionMd <=sCriticoD*-1)
    sAsimetricoDCuad=(relacionMd >= sCriticoD)

    #relacion interpierna Izquierda
    criticoI=(cuad_izq+isq_izq)*0.06
    sCriticoI=(cuad_izq+isq_izq)*0.3
    relacionMi=(cuad_izq+isq_izq)*0.7-cuad_izq

    simetricoMi=0
    asimetricoIIsq=0
    asimetricoICuad=0
    sAsimetricoIIsq=0
    sAsimetricoICuad=0

    simetricoMi=(relacionMi <=criticoI) & (relacionMi >=criticoI*-1)
    asimetricoIIsq=(relacionMi <criticoI*-1) & (relacionMi >=sCriticoI*-1)
    asimetricoICuad=(relacionMi >criticoI) & (relacionMi <sCriticoI)
    sAsimetricoIIsq=(relacionMi <=sCriticoI*-1)
    sAsimetricoICuad=(relacionMi >= sCriticoI)


    pdf.add_page()
    pdf.set_font("Arial",size=12)
    caratula("Club Hebraica Hockey", "Evaluación de la fuerza isométrica de MMII con celda de cargas")
    pdf.add_page()
    pdf.set_font("Arial",size=12, style="B")
    pdf.set_col(1)
    pdf.set_y(2)

    pdf.multi_cell(h=0.35, align='L', w=0, txt=f"1) FUERZA DE MUSLOS\n 1.1) RELACION FLEXORES (-10(ok)<10-25(leve)<+25(grave) ")
    if simetricoFlex !=0:
      pdf.cell(2,2,f'Valores Óptimos en flexores \n Porcentaje de asiemtría ={relacionFlex}',ln=False, align='l')
    elif asimetricoDFlex !=0:
      pdf.cell(2,2,f'Fortalecer flexores D \n Porcentaje de asiemtría ={relacionFlex}',ln=False, align='l')
    else:
      pdf.cell(2,2,f'Fortalecer flexores I \n Porcentaje de asiemtría ={relacionFlex}',ln=False, align='l')


    print("2.2) RELACION EXTENSORES (-10(ok)<10-25(leve)<+25(grave))")
    if simetricoExt !=0:
      print('Valores Óptimos en Extensores \n Porcentaje de asiemtría =',relacionExt)
    elif asimetricoDExt !=0:
      print('Fortalecer Extensores D \n Porcentaje de asiemtría =',relacionExt)
    else:
      print('Fortalecer Extensores I \n Porcentaje de asiemtría =',relacionExt)


    print("2.3) RELACION MUSLO DERECHO (-",criticoD,"(ok)<",criticoD,"-",sCriticoD,"(leve)<+",sCriticoD,"(grave))")
    if simetricoMd !=0:
      print('Valores Óptimos en Muslo derecho \n Porcentaje de relación =',relacionMd)
    elif asimetricoDIsq !=0 or sAsimetricoDIsq!=0:
      print('Fortalecer Flexores D \n Porcentaje de relación =',relacionMd)
    else:
      print('Fortalecer Extensores D \n Porcentaje de relación =',relacionMd)


    print("2.4) RELACION MUSLO Izquierdo (-",criticoI,"(ok)<",criticoI,"-",sCriticoI,"(leve)<+",sCriticoI,"(grave))")
    if simetricoMi !=0:
      print('Valores Óptimos en Muslo Izquierdo \n Porcentaje de relación =',relacionMi)
    elif asimetricoIIsq !=0 or sAsimetricoIIsq!=0:
      print('Fortalecer Flexores I \n Porcentaje de relación =',relacionMi)
    else:
      print('Fortalecer Extensores I \n Porcentaje de relación =',relacionMi)

    exportar(nombre)

    return

def exportar(nombre):
  pdfFileName = nombre +'.pdf'#NO EXPORTA EL PDF
  pdf.output(pdfFileName)
  shutil.copy(pdfFileName,"/content/drive/Shareddrives/Sociedad Hebraica Argentina/Noviembre 2023/Datos/Celda/Informes")
  return
lista=leer_carpeta(direccion_final+"Procesados/")
#print(lista)
for archivo in lista:
  df2=pd.read_csv(direccion_final+"Procesados/"+archivo)
  #print(df2)
  procesar_maximos(df2['cuad_der'][0],df2['cuad_izq'][0],df2['isq_der'][0],df2['isq_izq'][0],archivo[:-4])

2.2) RELACION EXTENSORES (-10(ok)<10-25(leve)<+25(grave))
Valores Óptimos en Extensores 
 Porcentaje de asiemtría = 2.234858713089878
2.3) RELACION MUSLO DERECHO (- 5.81592 (ok)< 5.81592 - 29.0796 (leve)<+ 29.0796 (grave))
Valores Óptimos en Muslo derecho 
 Porcentaje de relación = 3.4883999999999986
2.4) RELACION MUSLO Izquierdo (- 5.701560000000001 (ok)< 5.701560000000001 - 28.507800000000003 (leve)<+ 28.507800000000003 (grave))
Valores Óptimos en Muslo Izquierdo 
 Porcentaje de relación = 3.5612000000000066


KeyError: ignored

In [ ]:
pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"1) FUERZA DE MUSLOS\n 1.1) RELACION FLEXORES ")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Óptimos en isquios \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer isquios D \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer isquios I \n',ln=True, align='l')

pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoFlex])!=0:
  #pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data['nombre'][simetricoFlex]}")
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionFlex']][simetricoFlex]}\n")
#if len()!=0:

pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][asimetricoDFlex])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +10%  \n {data[['nombre','relacionFlex']][asimetricoDFlex]}  \n")

if len(data['nombre'][data['relacionFlex']<-25 *data['celda']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +25% \n  {data[['nombre','relacionFlex']][data['relacionFlex']<-25 *data['celda']]}\n")

pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][asimetricoIFlex])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +10% \n {data[['nombre','relacionFlex']][asimetricoIFlex]} \n  ")

if len(data['nombre'][data['relacionFlex'] >25 *data['celda']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +25% \n  {data[['nombre','relacionFlex']][data['relacionFlex'] >25 *data['celda']]}\n")

#relacion extensores
data['relacionExt']=(data['cuadDer']/data['cuadIzq'])*100-100
data['relacionExt']=data['relacionExt'].round(1)
data['relacionExt']=data['relacionExt']* data['celda']

simetricoExt=(data['relacionExt'] <=10) & (data['relacionExt'] >=-10) & data['celda']
asimetricoDExt=(data['relacionExt'] <-10) & (data['relacionExt'] >=-25) & data['celda']
asimetricoIExt=(data['relacionExt'] >10) & (data['relacionExt'] <25) & data['celda']

pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"2.2) RELACION EXTENSORES")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Óptimos en cuádriceps \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer cuádriceps D \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer cuádriceps I \n',ln=True, align='l')

pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoExt])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionExt']][simetricoExt]}")
pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][asimetricoDExt])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +10%  \n {data[['nombre','relacionExt']][asimetricoDExt]} \n ")
if len(data['nombre'][data['relacionExt']<-25 * data['celda']]) !=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +25% \n  {data[['nombre','relacionExt']][data['relacionExt']<-25 * data['celda']]}\n")
pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][asimetricoIExt])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +10% \n {data[['nombre','relacionExt']][asimetricoIExt]} \n ")
if len(data['nombre'][data['relacionExt'] >25 *data['celda']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +25% \n  {data[['nombre','relacionExt']][data['relacionExt'] >25 *data['celda']]}\n")


#relacion interpierna derecha
data['criticoD']=(data['cuadDer']+data['isqDer'])*0.06
data['sCriticoD']=(data['cuadDer']+data['isqDer'])*0.3
data['relacionMd']=(data['cuadDer']+data['isqDer'])*0.7-data['cuadDer']
data['relacionMd']=data['relacionMd'].round(1)
data['relacionMd']=data['relacionMd']* data['celda']

data['simetricoMd']=(data['relacionMd'] <=data['criticoD']) & (data['relacionMd'] >=data['criticoD']*-1) & data['celda']
data['asimetricoDIsq']=(data['relacionMd'] <data['criticoD']*-1) & (data['relacionMd'] >=data['sCriticoD']*-1) & data['celda']
data['asimetricoDCuad']=(data['relacionMd'] >data['criticoD']) & (data['relacionMd'] <data['sCriticoD']) & data['celda']
data['sAsimetricoDIsq']=(data['relacionMd'] <=data['sCriticoD']*-1) * data['celda']
data['sAsimetricoDCuad']=(data['relacionMd'] >= data['sCriticoD']) * data['celda']


pdf.add_page()
pdf.set_font("Arial",size=10, style="B")

pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"2.3) RELACION MUSLO DERECHO\n  ")


pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Óptimos inter pierna D \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer Isquios D \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer cuádriceps D \n',ln=True, align='l')

pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][data['simetricoMd']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionMd']][data['simetricoMd']]}\n")
#pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionMd','criticoD','sCriticoD']][data['celda']]}")

pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][data['asimetricoDIsq']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +6%  \n {data[['nombre','relacionMd']][data['asimetricoDIsq']]} \n  ")
if len(data['nombre'][data['sAsimetricoDIsq']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +30% \n {data[['nombre','relacionMd']][data['sAsimetricoDIsq']]}\n")

pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][data['asimetricoDCuad']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +6% \n {data[['nombre','relacionMd']][data['asimetricoDCuad']]} \n ")
if len(data['nombre'][data['sAsimetricoDCuad']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +30% \n {data[['nombre','relacionMd']][data['sAsimetricoDCuad']]}\n")

#relacion interpierna Izquierda
criticoI=(data['cuadIzq']+data['isqIzq'])*0.06

data['relacionMi']=(data['cuadIzq']+data['isqIzq'])*0.7-data['cuadIzq']
data['relacionMi']=data['relacionMi'].round(1)
data['relacionMi']=data['relacionMi']* data['celda']
#columnas=['nombre','relacionMi']
simetricoMi=(data['relacionMi'] <=criticoI) & (data['relacionMi'] >=criticoI*-1) & data['celda']
asimetricoIIsq=(data['relacionMi'] <criticoI*-1) & (data['relacionMi'] >=criticoI*-5) & data['celda']
asimetricoICuad=(data['relacionMi'] >criticoI) & (data['relacionMi'] <criticoI*5) & data['celda']
sAsimetricoIIsq=(data['relacionMi'] <=criticoI*-5) * data['celda']
sAsimetricoICuad=(data['relacionMi'] >= criticoI*5) * data['celda']

pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"2.4) RELACION MUSLO IZQUIERDO")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Óptimos en inter pierna I \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer Isquios I \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer cuádriceps I \n',ln=True, align='l')



pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoMi])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionMi']][simetricoMi]}\n")
pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][asimetricoIIsq])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +6%  \n {data[['nombre','relacionMi']][asimetricoIIsq]} \n")
if len(data['nombre'][sAsimetricoIIsq])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +30% \n {data[['nombre','relacionMi']][sAsimetricoIIsq]} \n")
pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][asimetricoICuad])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +6% \n {data[['nombre','relacionMi']][asimetricoICuad]} \n  ")
if len(data['nombre'][sAsimetricoICuad])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +30% \n {data[['nombre','relacionMi']][sAsimetricoICuad]} \n")


